# Linear Regression in Practice

In this lab we will work through an extended example of exploratory data analysis and supervised machine learning using the California Housing Price Dataset. This dataset consists of data about housing characteristics and prices in many districts of the state of California. The **task** this dataset asks us to solve is estimating the median house value in a district from a set of independent housing characteristics.

**Note**: the exercises are inline in this notebook and *not* at the end. The exercises will ask you to write some code and sometimes to provide some analysis of your findings in Markdown cells at the end of the exercise.

The main objectives of this laboratory are:
+ For you to gain familiarity with using Python for numerical programming in Jupyter Notebooks, Google Colaboratory, or *[INSERT TOOL OF CHOICE]*.
+ For you to gain familiarity with working with data in Numpy, Pandas, and Scikit-learn.
+ For you to learn how to use *visualization* as a tool for understanding the nature of machine learning problems and gain insight into potential solutions.
+ To learn to use Scikit-learn to solve simple univariate regression problems and validate your solutions.
+ To learn the value of *encapsulation* and *abstraction* of pipeline code for making experiments *reproducible*.

---
---

# CODE OF CONDUCT V1.0

This Code of Conduct outlines principles for the ethical and transparent use of Large Language Models (LLMs) and existing and internet resources, ensuring integrity and accountability in your laboratory submissions. This first version of the FML Laboratory Code of Conduct was developed in a brainstorming session with **ChatGPT Version 2**, modifying its proposal to the specifics of the FML Laboratories and concentrating on the transparent collaboration with classmates and transparent use of LLMs.

#### **1. Transparency in LLM Use**
- **Clear Disclosure:** Explicitly state when Large Language Models (LLMs) are used in any part of the process, including data analysis, code generation, or writing.
- **Model Limitations:** Acknowledge the inherent limitations of LLMs, such as potential biases, and make clear where human intervention was applied to verify results or to modify/augment produced code.

#### **2. Proper Attribution and Documentation**
- **Attribution:** Provide appropriate citations and credits for all external resources, including code, data, and models.
- **Clear Documentation:** Maintain detailed records of tools, methods, and models used, ensuring transparency and reproducibility in your submitted laboratory solutions.

#### **3. Collaboration and Individual Work**
- **Sharing Solutions:** While collaboration and discussing ideas with classmates are encouraged, solutions to assignments or projects should be your own. Do not copy or submit work created by others, including code or models, as your own.
- **Submission Integrity:** All submitted work must reflect your own understanding and effort. If external tools, LLMs, online resources, or code from your classmates were used, they must be properly documented, but the final submission must be an individual effort.

#### **4. Accountability**
Non-compliance with these guidelines will be subject to review by the course exam commission, with possible disciplinary actions.

---
---

## Part 1: Warming Up

In this first set of exercises we will analyze our dataset and build a simple linear regression pipeline. This is a fairly typical task that is asked of anyone working with Data Science: Here is some data, do something useful with it!

### Step 1: Data Modeling

OK, let's get started. The first thing we want to do is get our dataset loaded and start to get a feel for it. This is always a good idea -- we *play* with the data first in order to get a better understanding of it.

In [2]:
# Initial imports -- these are fairly standard.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Import the function that will download the dataset.
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the sklearn version of the California Housing dataset.
ds = fetch_california_housing()

### Exercise 1a: Poking Around

Spend some time looking at the elements of the `ds` we just loaded (it's a python `dict`). Find the description of the dataset and make sure you understand what the features are and what the targets variable is. **Hints**: to get the keys of the dictionary, use: `ds.keys()`.

We are going to construct a Pandas `DataFrame` in the next exercise. Where can you get reasonable column names from the sklearn dataset object?

In [ ]:
# Your code here.

print(ds.keys())
print(ds.DESCR)
print(ds.data.shape)   
print(ds.feature_names)
print(ds.target.shape)
print(ds.target_names)
print(ds.data)

**Your Analysis**: We have a regression problem (see documentation in 'ds.DESCR') with 8 indipendent input varianles (see names in 'ds.feature_name') ad a single dependent target (see 'ds.target_names'). The dataset contains 20k+ rows pf samples, our goal is to estimate the median house value for California housing distrincts from the given household variables.

scrivere ciò che si scopre via via.

### Exercise 1b: Creating a Pandas DataFrame

OK, now we can create the `DataFrame` to hold our independent variables and a `Series` to hold the target values. Make sure you use good column names when constructing the `DataFrame`. Some relevant documentation: [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) and [pandas.Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html).

In [ ]:
# Create a Pandas DataFrame for our dataset and a Pandas Series for the targets.

# Your code to build the DataFrame here (replace None)
df = pd.DataFrame(data = ds.data, columns = ds.feature_names)
print(df)

# Your code to build the target Series here (replace None)
targets = pd.Series(ds.target)
print(targets)

#pandas serve per creare delle righe e colonne con i dati che abbiamo dandogli un nome

### Exercise 1c: Examining the Data

Study the *descriptive statistics* of the data. Do you notice anything "strange" about any of the features? Are the features scaled similarly? **Hint**: Use the `.describe()` method on the DataFrame you created.

In [ ]:
# Your code here.
df.describe()


**Your Analysis Here** (in Markdown).
We do not have any missing values, most features are in the $\mathcal(0)(1)$ ange, with some several orders of magnitude higher (Population, in particular, is in the $\mathcal(0)(1000)$ rage). The variance of input features are generally in range with range with their magnitudes, with the exception of AveOccup which has an elevated variance. 

Non ci sono dati mancanti.

In [ ]:
targets.describe()

**Analysis**: our target variables are distributed reasonably well (at least in the first-and second-order sense), which mean and standard deviation in the same range.

---
### Step 2: Visualization

OK, now that we have a bit of a *feel* for our data, let's get a better idea about it through visualization.

### Exercise 2a: Visualizing the Target
Create a plot to study the **distribution** of our target values. The best tool for that is a **histogram**. Search for this functionality in the Matplotlib documentation.

**Note**: In addition to *histograms*, try out the Seaborn function `distplot`.

In [ ]:
# Your visualization code here.
targets.hist(bins=16, edgecolor='black') #pandas ha un metodo hist che permette di fare un istogramma


mettere un po' di caxzzate
**Your Analysis Here**.

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(targets, bins=16, edgecolor='black')

plt.show()

plt.figure(figsize=(10, 8))
sns.histplot(targets, bins=16, kde=True, color='lightblue')

plt.title('Targets')
plt.show()

**Analysis:** Kind of a skewed Gaussian shape for our target values, with a long-is tail 

### Exercise 2b: Subplots
Now create a multi-plot figure to visualize the distributions of **all** of the independent features in the dataset. Make sure you use `figsize=` to resize the figure appropriately.

A few things that will help with this:
+ If you want to index columns by **integer** indices, use the `.iloc()` method (e.g. `df.iloc[:,1]`).
+ If you extract a column as a `Series` from a `DataFrame`, you can recover its name with the `name` attribute.
+ Encapsulate you plotting code in a **function** you can call later.

**Super Hint**: Pandas already has this functionality **built-in**. If you can find it, use it!


In [ ]:
df.MedInc.hist(bins=50, edgecolor='black') 
plt.figure(figsize=(10, 8))
df.Population.hist(bins=50, edgecolor='black') #pandas ha un metodo hist che permette di fare un istogramma

In [ ]:
plt.figure(figsize=(15, 15))
for (i, col) in enumerate(df.columns):
    plt.subplot(3, 3, i+1)  # plot on grid 3x3
    plt.hist(df[col], 30, edgecolor='white', color='lightblue')
    plt.title(df[col].name)


**Analysis:** We can see a spike at the start or AveBedrms, Population and AveOccup.
We can use the Pandas methods or the matplotlib function, the result is pretty similar. 

Se l'istogramma è così scalato vuol dire che un valore a dx ci sarà anche in quelli che non si vedono. (mia)

**Analysis:** (prof) Several input features are drammatically skewed due to outliers (this is documented in the ds.DESCR). Other more reasonable, with Gaussian or simple binomial distribution (e.g. Latitude and Longitude). 

In [ ]:
plt.scatter(df.Longitude, df.Latitude, c=targets, cmap='hot') 

**Analysis:** Use covarianza


---
---
## Step 3: Split you Data

A very important step. Now we will split our `DataFrame` into training and testing splits.

### Exercise 3.1: Create a Split
Now we need to create our training and testing splits. Read the documentation for `sklearn.model_selection.train_test_split()`. Use this function to create a **training** split with 75% of the data, and a **test** split with 25% of the data.

In [ ]:
# Your code here.
from sklearn.model_selection import train_test_split

# Split data into 75-25 train/test split -- replace the [None]*4 with your code.
(Xtr, Xte, ytr, yte) = train_test_split(df, targets, test_size=0.25) #tutte le volte io cambio i valori che vado a prendere, per evitare questo si può usare un seed

print("Test Set shape:", Xte.shape)
print("Training Set shape:", Xtr.shape)

**Note**: *My* convention for data and label matrices is `Xtr, ytr` (training) and `Xte, yte` (testing) because the `X` and `y` helps link the *code* to the *math*. Start developing your own style to help you organize (and eventually share) your code.

### Exercise 3.2: Fit a LinearRegression
Finally some machine learning. Study the documentation for `class sklearn.linear_model.LinearRegression`. Then write some code to fit a linear regression model to your **training** split. Try out your model by computing predictions on some data (use the `model.predict()` method).


In [ ]:
from sklearn.linear_model import LinearRegression

## 1. Creare il modello di regressione lineare
model = LinearRegression()

# 2. Addestrare il modello sui dati di training
model.fit(Xtr, ytr)

# 3. Fare previsioni sui dati di test
y_pred = model.predict(Xte)

model.coef_

model.intercept_
# Stampare le previsioni
print("Previsioni sui dati di test:", y_pred)
print("coef:", model.coef_)
print("intercept", model.intercept_)

print("sum", model.coef_ @ Xte.T + model.intercept_) #numpy @ è il prodotto matriciale

### Exercise 3.3: Evaluate your Model
Write some code to compute the root mean-squared error (RMSE) and mean absolute error (MAE) for you model predictions. Try it on both the **test** and **training** splits.

In [14]:
def mae(y_true, y_pred):
    return (y_true - y_pred).abs().mean()

def rmse(y_true, y_pred):
    return np.sqrt(((y_true - y_pred)**2).mean())

In [ ]:
preds_train = model.predict(Xtr)
preds_test = model.predict(Xte)

print(f'MAE on Train: {mae(ytr, preds_train)}')
print(f'RMSE on Train: {rmse(ytr, preds_train)}')
print(f'MAE on Test: {mae(yte, preds_test)}')
print(f'RMSE on Test: {rmse(yte, preds_test)}')


**Your Analysis Here**: Why is the performance on the train set different than that on the test split? What if you change the proportion of training to test data in your splits?

### Exercise 3.4: Visualizing the Results
Now I want you to write a function that makes a **residual plot** of the data and the model predictions. This plot should show, for each data point, the **signed error** (i.e. y - predicted) of the model prediction. Do you notice any **patterns** in the errors? Can you link this to previous analyses you made? 

In [ ]:
(Xtr, Xte, ytr, yte) = train_test_split(df, targets, train_size=0.5)

model = LinearRegression()
model.fit(Xtr, ytr)


def resplot(y, preds):
    plt.scatter(preds, preds-y, s=0.3)
    plt.title("Residual Plot")  
    plt.xlabel("Predictions")
    plt.ylabel("Residuals")


resplot(model.predict(df), targets)

**Your Analysis Here**

### Step 4: Repeat.

Now you should put all of the pieces together into a repeatable, reproducible pipeline.

### Exercise 4: The Pipeline
Write a function (or even just code in the cell that calls previously defined functions) that runs an **experiment**:
1. Splitting data
1. Instantiating the model
1. Fitting the model
1. Evaluating the model
1. (Maybe) Visualizing results

Experiment with different splits to see if the results are the same. Try using more or less training data with respect to test data. Observe how the results change.

In [ ]:
# Your pipeline code here.
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def pipeline(df, targets, train_size=0.75, degree = 1):
    
    (Xtr, Xte, ytr, yte) = train_test_split(df, targets, test_size=0.25)
    
    model = LinearRegression()
    model.fit(Xtr, ytr)
    
    preds_tr = model.predict(Xtr)
    preds_te = model.predict(Xte)
    
    rmse_train = np.square(np.subtract(ytr, preds_tr)).mean()
    rmse_test = np.square(np.subtract(yte, preds_te)).mean()
    mae_train = np.abs(np.subtract(ytr, preds_tr)).mean()
    mae_test = np.abs(np.subtract(yte, preds_te)).mean()
    
    print(f'Train size: {train_size}')
    print(f'Degree: {degree}')
    print(f'RMSE on train: {rmse_train:.4f}')
    print(f'RMSE on test: {rmse_test:.4f}')
    print(f'MAE on train: {mae_train:.4f}')
    print(f'MAE on test: {mae_test:.4f}')
    print('--------')
    
    # Plot the data
    plt.figure(figsize=(12, 6))
    plt.scatter(ytr, preds_tr, label='Train Data', alpha=0.5)
    plt.scatter(yte, preds_te, label='Test Data', alpha=0.5)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title(f'Degree {degree}, Train Size {train_size}')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    
    return rmse_train, rmse_test, mae_train, mae_test

for train_size in range(1, 10, 2):
    train_size /= 10.0  # Convert to a float in the range [0.0, 1.0]
    pipeline(df, targets, train_size)
    print("Train size:", train_size)
    print('--------')


#https://www.geeksforgeeks.org/python-mean-squared-error/
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html

**Your Analysis Here**: Experiment with different splits to see if the results are the same. Try using more or less training data with respect to test data. Observe how the results change.

---
---
## Part 2: Improving our Regressor

Now that we have a simple, baseline linear regression result, let's see if we can't improve on it. This is where the real work begins, and where it is **super** important to ensure that the conclusions we draw are *valid*.

**Questions**: Are our independent variables *scaled* similarly? Does our model have *high variance* -- that is, if we fit it to a new training sample, does the result vary dramatically?

### Exercise 5: Increasing Model Capacity

Check out the documentation for `sklearn.preprocessing.PolynomialFeatures`. Map the independent variables onto a **polynomial** basis of variable order. Fit your model using your pipeline from above and observe its behavior for different degree polynomial embeddings.

In [ ]:
# Your code here.

from sklearn.preprocessing import PolynomialFeatures

rmse_train_values = []
rmse_test_values = []
mae_train_values = []
mae_test_values = []

from sklearn.preprocessing import PolynomialFeatures

def variableSizeModels(train_size = 0.75):
    for degree in range(1, 4):
        
        poly = PolynomialFeatures(degree)    # Create PolynomialFeatures instance with the current degree

        df_poly = poly.fit_transform(df)   # Apply the PolynomialFeatures instance to the dataframe
        
        rmse_train, rmse_test, mae_train, mae_test = pipeline(df_poly, targets, train_size=train_size , degree=degree) 
        
        rmse_train_values.append(rmse_train)
        rmse_test_values.append(rmse_test)
        mae_train_values.append(mae_train)
        mae_test_values.append(mae_test)
                

variableSizeModels()

#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

cazzate


### Exercise 6: Hyperparameter Selection and Cross-validation

How should we select the correct *degree* for our polynomial basis? Is the performance on the *training* set equal to the performance on the *test* set? Why?

In [ ]:
# Your code here.

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np



def variableSizeModels2(train_size = 0.75):
    
    
    for degree in range(1, 5):
        
        poly = PolynomialFeatures(degree)    # Create PolynomialFeatures instance with the current degree

        df_poly = poly.fit_transform(df)   # Apply the PolynomialFeatures instance to the dataframe
    
        (Xtr, Xte, ytr, yte) = train_test_split(df_poly, targets, train_size=train_size)    
        
        model = LinearRegression()
        model.fit(Xtr, ytr)
        
        #----------
        # Perform k-fold cross-validation and calculate mean squared error (MSE)
        
        mse_scores = cross_val_score(model, Xtr, ytr, scoring =  "neg_mean_squared_error")
        print("Current degree:", degree)
        
        mse_scores = -mse_scores

        for fold, mse in enumerate(mse_scores, 1):
            print(f'Fold {fold}: MSE = {mse:.4f}')
            
        average_mse = np.mean(mse_scores)
        print(f'Average MSE: {average_mse:.4f}')
        
        #---------
        
        param_grid = {
            'fit_intercept': [True, False],
        }        
        #When fit_intercept is set to True, the model will estimate both the slope (coefficients) and the intercept.
        #When fit_intercept is set to False, the model assumes that the intercept is zero.

        # class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None,
        # refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
        
        grid_search = GridSearchCV(model, param_grid, scoring="neg_mean_squared_error")

        grid_search.fit(Xtr, ytr)

        best_params = grid_search.best_params_
        print(f'Best Hyperparameters:  {best_params}')

        # Get the best estimator (model) from the grid search
        best_model = grid_search.best_estimator_

        # Evaluate the best model on the test data
        mse = -grid_search.score(Xte, yte)
        print(f'Best Model MSE on Test Data: {mse:.4f}')
        print('--------')
                

variableSizeModels2()


**Your Final Analysis Here**: Summarize the conclusions you can make about the best hyperparameter settings for this dataset. How do you know your conclusions are supported by the data?

---
---

## Part 3: Optional

### Exercise 7 (BONUS): Ordinary Least Squares with Gradient Descent
This is an *optional* exercise, but I think everyone should implement gradient descent at least once in their life, so if you like a challenge this assignment can earn you up to five (5) bonus points our of 100 on this assignment.

#### Exercise 7.1: The Loss Function
Recall that the loss function for parameters $\mathbf{w}$ on dataset $\mathcal{D} = \{ (\mathbf{x}_n, y_n)\}_{n=1}^{N}$ we use for least squares regression is (assume that the inputs $\mathbf{x}$ have already been augmented with $x_0 = 1$ to account for the bias parameter $w_0$):
$$
\mathcal{L}(\mathcal{D}, \mathbf{w}) = \frac{1}{2} \sum_{n=1}^{N} (\mathbf{w}^T \mathbf{x}_n - y_n)^2
$$

Start by implementing a function `error(w, Xs, ys)` that computes this loss function. This function should be *independent* of feature dimensionality and **do not use loops**!

Test this function using data from the California Housing dataset (after augmenting it with a constant 1 in the first dimension!) with the parmaters $\mathbf{w}$ set to the solution found by `LinearRegression`. Compare it to randomly initialized $\mathbf{w}$.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Define the error function
def error(w, Xs, ys):
    # Compute the predictions for all data points
    predictions = np.dot(Xs, w)
    
    # Compute the residuals (predicted - actual values)
    residuals = predictions - ys
    
    # Compute the squared error and take the sum, then divide by 2
    loss = 0.5 * np.sum(residuals ** 2)
    
    return loss

# Load California housing dataset
data = fetch_california_housing()
X = data.data
y = data.target

# Add a bias term (column of 1s) to the design matrix X
X = np.hstack([np.ones((X.shape[0], 1)), X])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Fit the LinearRegression model
model = LinearRegression()
model.fit(X_train, y_train)

# Use only the coefficients (since the bias is included in the X matrix)
w_learned = model.coef_

# Compute the loss using the learned weights
train_loss_learned = error(w_learned, X_train, y_train)
test_loss_learned = error(w_learned, X_test, y_test)

print(f'Learned weights - Training Loss: {train_loss_learned:.4f}, Test Loss: {test_loss_learned:.4f}')

# Compare with randomly initialized weights
np.random.seed(42)
w_random = np.random.randn(X_train.shape[1])

train_loss_random = error(w_random, X_train, y_train)
test_loss_random = error(w_random, X_test, y_test)

print(f'Random weights - Training Loss: {train_loss_random:.4f}, Test Loss: {test_loss_random:.4f}')


#### Exercise 7.2: The Gradient of the Loss Function
Now we need the *gradient* of the loss function in order to improve our solution. Recall that the gradient of the loss is:
$$
\begin{eqnarray*}
\nabla_{\mathbf{w}} \mathcal{L}(\mathcal{D}, \mathbf{w}) &=& \nabla_{\mathbf{w}} \frac{1}{2} \sum_{n=1}^{N} (\mathbf{w}^T \mathbf{x}_n - y_n)^2 \\
&=& \frac{1}{2} \sum_{n=1}^{N} \nabla_{\mathbf{w}} (\mathbf{w}^T \mathbf{x}_n - y_n)^2 \mbox{ (by linearity of the gradient)} \\
&=& \sum_{n=1}^{N} (\mathbf{w}^T \mathbf{x}_n - y_n) \nabla_{\mathbf{w}} (\mathbf{w}^T \mathbf{x}_n) \mbox{ (by gradient rule for quadratics and chain rule)}
\end{eqnarray*}
$$
Write a Python function `grad_error(w, Xs, ys)` to compute this gradient of the loss function for a given $\mathbf{w}$ and dataset $\mathcal{D}$. Again, if you find yourself writing loops, your probably doing something wrong!

In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Define the gradient of the error function
def grad_error(w, Xs, ys):
    # Compute the residual (predicted - actual values)
    residuals = np.dot(Xs, w) - ys
    
    # Compute the gradient as X^T * residuals
    grad = np.dot(Xs.T, residuals)
    
    return grad

# Load California housing dataset
data = fetch_california_housing()
X = data.data
y = data.target

# Add a bias term (column of 1s) to the design matrix X
X = np.hstack([np.ones((X.shape[0], 1)), X])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Fit the LinearRegression model
model = LinearRegression()
model.fit(X_train, y_train)

# Use only the coefficients (since the bias is included in the X matrix)
w_learned = model.coef_

# Compute the gradient using the learned weights
grad_learned = grad_error(w_learned, X_train, y_train)
print("Gradient using learned weights:")
print(grad_learned)

# Compare with randomly initialized weights
np.random.seed(42)
w_random = np.random.randn(X_train.shape[1])

# Compute the gradient using random weights
grad_random = grad_error(w_random, X_train, y_train)
print("\nGradient using random weights:")
print(grad_random)


#### Exercise 7.3: Gradient Descent
Now we have everything we need to, starting from a randomly initialized $\mathbf{w}$ and some data $\mathcal{D}$, iteratively improve our solution (up to some numerical tolerance). Recall that our gradient descent update rule is:
$$
\mathbf{w}_{t+1} = \mathbf{w}_t - \eta \nabla_{\mathbf{w}} \mathcal{L}(\mathcal{D}, \mathbf{w}_{t})
$$
Write a Python function `gradient_descent(w, Xs, ys, eta, tol=1e-5)` to perform gradient descent to convergence (i.e. until the total loss between iterations is less than `tol`). Here you can use a loop.

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Gradient of the error function
def grad_error(w, Xs, ys):
    residuals = np.dot(Xs, w) - ys
    grad = np.dot(Xs.T, residuals)
    return grad

# Error function (loss)
def error(w, Xs, ys):
    predictions = np.dot(Xs, w)
    residuals = predictions - ys
    loss = 0.5 * np.sum(residuals ** 2)
    return loss

# Gradient Descent Algorithm with Gradient Clipping
def gradient_descent(w, Xs, ys, eta, tol=1e-5, max_iter=1000, clip_value=100):
    prev_loss = error(w, Xs, ys)
    for i in range(max_iter):
        # Compute the gradient of the loss
        grad = grad_error(w, Xs, ys)
        
        # Clip the gradient to avoid overflow
        grad = np.clip(grad, -clip_value, clip_value)
        
        # Update the weights using the gradient
        w = w - eta * grad
        
        # Compute the new loss
        curr_loss = error(w, Xs, ys)
        
        # Check for convergence (if loss improvement is below tolerance)
        if abs(prev_loss - curr_loss) < tol:
            print(f'Converged after {i+1} iterations.')
            break
        
        prev_loss = curr_loss

    return w

# Load California housing dataset
data = fetch_california_housing()
X = data.data
y = data.target

# Normalize the data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Add a bias term (column of 1s) to the design matrix X
X_scaled = np.hstack([np.ones((X_scaled.shape[0], 1)), X_scaled])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

# Initialize weights with smaller random values
np.random.seed(42)
w_random = np.random.randn(X_train.shape[1]) * 0.01  # Smaller initialization

# Run gradient descent with a smaller learning rate and gradient clipping
learning_rate = 1e-6  # Smaller learning rate
w_optimized = gradient_descent(w_random, X_train, y_train, eta=learning_rate)

# Output the final optimized weights
print("Optimized weights from gradient descent:")
print(w_optimized)


Now use your implementation of gradient descent to solve the regression problem for the California Housing dataset. You will probably have to play with different learning rates, but verify that you end up with a solution (very) close to the one found by Scikit-learn's `LinearRegression` class.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Load California housing dataset
data = fetch_california_housing()
X = data.data
y = data.target

# Normalize the data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Add a bias term (column of 1s) to the design matrix X
X_scaled = np.hstack([np.ones((X_scaled.shape[0], 1)), X_scaled])  # Add intercept term

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

# Implement gradient descent functions
def grad_error(w, Xs, ys):
    residuals = np.dot(Xs, w) - ys
    grad = np.dot(Xs.T, residuals)  # Gradient of the loss function
    return grad

def error(w, Xs, ys):
    predictions = np.dot(Xs, w)
    residuals = predictions - ys
    loss = 0.5 * np.sum(residuals ** 2)  # Loss function
    return loss

def gradient_descent(w, Xs, ys, eta, tol=1e-5, max_iter=1000, clip_value=100):
    prev_loss = error(w, Xs, ys)
    for i in range(max_iter):
        # Compute the gradient of the loss
        grad = grad_error(w, Xs, ys)
        
        # Clip the gradient to avoid overflow
        grad = np.clip(grad, -clip_value, clip_value)
        
        # Update the weights using the gradient
        w = w - eta * grad
        
        # Compute the new loss
        curr_loss = error(w, Xs, ys)
        
        # Check for convergence (if loss improvement is below tolerance)
        if abs(prev_loss - curr_loss) < tol:
            print(f'Converged after {i+1} iterations.')
            break
        
        prev_loss = curr_loss

    return w

# 1. Scikit-learn's LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

# Get the weights from Scikit-learn's LinearRegression
w_sklearn = np.hstack([model.intercept_, model.coef_])  # includes intercept
print("Scikit-learn Linear Regression weights:")
print(w_sklearn)
print(f"Shape of Scikit-learn weights: {w_sklearn.shape}")

# 2. Our Gradient Descent implementation
np.random.seed(42)
w_random = np.random.randn(X_train.shape[1]) * 0.01  # Initialize with a small random value for all weights (10 weights)

# Choose a learning rate
learning_rate = 1e-5
w_optimized = gradient_descent(w_random, X_train, y_train, eta=learning_rate)

print("Our Gradient Descent optimized weights:")
print(w_optimized)
print(f"Shape of Optimized weights: {w_optimized.shape}")

# Compare the two sets of weights
print("Difference between weights:")
try:
    print(w_optimized - w_sklearn)
except ValueError as e:
    print("Error during weight comparison:", e)

# Evaluate both models using Mean Squared Error (MSE)
y_pred_sklearn = model.predict(X_test)
mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
print(f"Scikit-learn MSE: {mse_sklearn:.4f}")

y_pred_gd = np.dot(X_test, w_optimized)
mse_gd = mean_squared_error(y_test, y_pred_gd)
print(f"Gradient Descent MSE: {mse_gd:.4f}")
